In [2]:
import pandas as pd
import numpy as np

In [3]:
uitslagenDF = pd.read_csv('Uitslag_alle_gemeenten_TK20210317.csv', sep=';')
uitslagenDF

,RegioNaam,RegioCode,AmsterdamseCode,OuderRegioNaam,OuderRegioCode,Kiesgerechtigden,Opkomst,OngeldigeStemmen,BlancoStemmen,GeldigeStemmen,...,OPRECHT,JEZUS LEEFT,DE FEESTPARTIJ (DFP),U-Buntu Connected Front,Vrij en Sociaal Nederland,Partij van de Eenheid,Wij zijn Nederland,Partij voor de Republiek,Modern Nederland,De Groenen
0,Aa en Hunze,G1680,10787,Assen,K3,20921,17444,32,25,17387,...,9.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
1,Aalsmeer,G0358,11264,Haarlem,K10,23249,19563,23,35,19505,...,11.0,NaN,12.0,2.0,5.0,NaN,2.0,NaN,NaN,NaN
2,Aalten,G0197,11046,Arnhem,K7,21386,17215,18,16,17181,...,4.0,12.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,Achtkarspelen,G0059,10199,Leeuwarden,K2,21772,16976,20,27,16929,...,15.0,16.0,NaN,NaN,0.0,NaN,NaN,4.0,NaN,2.0
4,Alblasserdam,G0482,11327,Dordrecht,K14,15070,12203,25,16,12162,...,7.0,11.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,Zundert,G0879,11192,Tilburg,K17,17244,13451,23,24,13404,...,4.0,4.0,5.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN
351,Zutphen,G0301,10254,Arnhem,K7,37888,30290,51,39,30200,...,12.0,19.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
352,Zwartewaterland,G1896,10746,Zwolle,K4,16766,14382,38,10,14334,...,3.0,10.0,NaN,2.0,NaN,1.0,1.0,NaN,NaN,NaN
353,Zwijndrecht,G0642,10468,Dordrecht,K14,34589,26030,50,47,25933,...,13.0,26.0,NaN,NaN,NaN,NaN,4.0,NaN,3.0,NaN


In [42]:
testDF = pd.DataFrame(data=list(uitslagenDF['OPRECHT']), columns=['aantal stemmen'],
                                index=list(uitslagenDF['RegioNaam']))

testDF.astype(int)

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [6]:
aantalZetels = 150
totaalStemmen = uitslagenDF['GeldigeStemmen'].sum()
kiesdeler = int(totaalStemmen / aantalZetels + 0.5)

In [33]:
zetelsDF = pd.DataFrame(columns=['totaal aantal stemmen', 'aantal zetels'],
                        index=uitslagenDF.columns[10:])

for partij in uitslagenDF.columns[10:]:
    partijStemmen = int(uitslagenDF[partij].sum())
    zetelsDF.loc[partij, 'totaal aantal stemmen'] = partijStemmen
    
    aantalVolleZetels = int(partijStemmen/kiesdeler)
    if aantalVolleZetels > 0:
        zetelsDF.loc[partij, 'aantal zetels'] = aantalVolleZetels
    else:
        zetelsDF = zetelsDF.drop(partij)
    
zetelsDF.insert(2, 'stemmen per zetel', [0]*len(zetelsDF.index))
zetelsDF

,totaal aantal stemmen,aantal zetels,stemmen per zetel
VVD,2279126,32,0
D66,1565862,22,0
PVV (Partij voor de Vrijheid),1124482,16,0
CDA,990601,14,0
SP (Socialistische Partij),623371,8,0
Partij van de Arbeid (P.v.d.A.),597192,8,0
GROENLINKS,537308,7,0
Forum voor Democratie,523083,7,0
Partij voor de Dieren,399751,5,0
ChristenUnie,351275,5,0


In [6]:
restzetels = 150 - zetelsDF['aantal zetels'].sum()
zetelsDF['stemmen per zetel'] = zetelsDF['totaal aantal stemmen'] / (zetelsDF['aantal zetels'] + 1)
zetelsDF['stemmen per zetel'] = pd.to_numeric(zetelsDF['stemmen per zetel'])

while restzetels > 0:
    imax = zetelsDF['stemmen per zetel'].idxmax() # index (partijnaam) van hoogste aantal stemmen per zetel
    zetelsDF.loc[imax, 'aantal zetels'] += 1
    zetelsDF.loc[imax, 'stemmen per zetel'] = zetelsDF.loc[imax, 'totaal aantal stemmen'] / (zetelsDF.loc[imax, 'aantal zetels'] + 1)
    restzetels -= 1

zetelsDF = zetelsDF.drop('stemmen per zetel', axis=1) 

zetelsDF

,totaal aantal stemmen,aantal zetels
VVD,2279126,34
D66,1565862,24
PVV (Partij voor de Vrijheid),1124482,17
CDA,990601,15
SP (Socialistische Partij),623371,9
Partij van de Arbeid (P.v.d.A.),597192,9
GROENLINKS,537308,8
Forum voor Democratie,523083,8
Partij voor de Dieren,399751,6
ChristenUnie,351275,5


In [10]:
type(zetelsDF.to_html())

str

In [8]:
len(zetelsDF)

17

In [13]:
f'{aantalZetels}'

'150'